In [1]:
import os
import locale
import numpy as np
import pandas as pd
import requests as r
from locale import atof
locale.setlocale(locale.LC_NUMERIC, '')

import numpy as np
np.printoptions(precision=2)

In [2]:
def fetch_data(url):
    df = pd.read_html(url)[0]
#     df = pd.read_csv("static/corona.csv", index_col=False)
    if len(df.columns) >= 8:
        df.drop(df.columns.values[-1], axis=1, inplace=True)
        new_col_names = ["Country", "Cases", "NewPatients", "Deaths", "RecentDeaths",
                         "Recovered", "ActivePatients", "CriticalPatients"]
        rename_dict = dict(zip(df.columns.values, new_col_names))
        df = df.rename(columns=rename_dict)
        df["NewPatients"] = df["NewPatients"].apply(lambda x: x is not None and type(x) is not float and '+' in x and atof(x.split('+')[1]))
        df["NewPatients"] = df["NewPatients"].replace(False, 0)
        df.drop(df[df["Country"].apply(lambda x: "Total" in x)].index.values, inplace=True)
        df.fillna(value=0, inplace=True)
        return df
    return {"code": 404}

In [3]:
res = r.get("https://www.worldometers.info/coronavirus/") 
df = fetch_data(res.text)
df.head()

,Country,Cases,NewPatients,Deaths,RecentDeaths,Recovered,ActivePatients,CriticalPatients
0,China,80881,21.0,3226.0,13.0,68709.0,8946,3226.0
1,Italy,27980,0.0,2158.0,0.0,2749.0,23073,1851.0
2,Iran,16169,1178.0,988.0,135.0,5389.0,9792,0.0
3,Spain,11409,1467.0,499.0,157.0,1028.0,9882,563.0
4,S. Korea,8320,84.0,81.0,6.0,1401.0,6838,59.0


In [4]:
workdf = df.sort_values("Cases", ascending=0).head(30)

In [5]:
country, activePatient, recovered, recoveryRate, deathRate = df["Country"], df["ActivePatients"], df["Recovered"], (df["Recovered"] / df["Cases"]) * 100, (df["Deaths"]/ df["Cases"]) * 100

In [6]:
data = {"Country": country,
        "ActivePatient": activePatient,
        "Recovered": recovered,
        "RecoveryRate": recoveryRate.apply(lambda x: round(x)),
        "DeathRate": deathRate.apply(lambda x: round(x))}

In [7]:
work = pd.DataFrame(data)

In [8]:
work.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163 entries, 0 to 162
Data columns (total 5 columns):
Country          163 non-null object
ActivePatient    163 non-null int64
Recovered        163 non-null float64
RecoveryRate     163 non-null int64
DeathRate        163 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 7.6+ KB


In [9]:
work = work.fillna(value=0)

In [10]:
work.head()

,Country,ActivePatient,Recovered,RecoveryRate,DeathRate
0,China,8946,68709.0,85,4
1,Italy,23073,2749.0,10,8
2,Iran,9792,5389.0,33,6
3,Spain,9882,1028.0,9,4
4,S. Korea,6838,1401.0,17,1


In [11]:
fresh = work[work["ActivePatient"] > 0][ work["RecoveryRate"] > 0]

/home/alpha/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [14]:
sort_by_RR = fresh.sort_values("RecoveryRate", ascending=0)

In [15]:
sort_by_RR.head()

,Country,ActivePatient,Recovered,RecoveryRate,DeathRate
0,China,8946,68709.0,85,4
99,Macao,3,10.0,77,0
17,Diamond Princess,233,456.0,66,1
91,Oman,12,12.0,50,0
42,Hong Kong,77,81.0,50,2


In [16]:
df.iloc[94]

Country             Dominican Republic
Cases                               21
NewPatients                          0
Deaths                               1
RecentDeaths                         0
Recovered                            0
ActivePatients                      20
CriticalPatients                     0
Name: 94, dtype: object

In [17]:
sort_by_RR.sort_values("DeathRate", ascending=0)

,Country,ActivePatient,Recovered,RecoveryRate,DeathRate
51,San Marino,89,4.0,4,9
1,Italy,23073,2749.0,10,8
65,Algeria,45,10.0,17,8
43,Iraq,102,41.0,27,7
36,Philippines,170,5.0,3,6
...,...,...,...,...,...
99,Macao,3,10.0,77,0
54,Mexico,78,4.0,5,0
46,Saudi Arabia,127,6.0,5,0
92,Tunisia,23,1.0,4,0


In [18]:
di = sort_by_RR.head().to_dict()

In [19]:
di.keys()

dict_keys(['Country', 'ActivePatient', 'Recovered', 'RecoveryRate', 'DeathRate'])

In [22]:
sort_by_RR.reset_index(drop=True, inplace=True)

In [24]:
sm = sort_by_RR.head()

In [25]:
sm.to_html(index=False)

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Country</th>\n      <th>ActivePatient</th>\n      <th>Recovered</th>\n      <th>RecoveryRate</th>\n      <th>DeathRate</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>China</td>\n      <td>8946</td>\n      <td>68709.0</td>\n      <td>85</td>\n      <td>4</td>\n    </tr>\n    <tr>\n      <td>Macao</td>\n      <td>3</td>\n      <td>10.0</td>\n      <td>77</td>\n      <td>0</td>\n    </tr>\n    <tr>\n      <td>Diamond Princess</td>\n      <td>233</td>\n      <td>456.0</td>\n      <td>66</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <td>Oman</td>\n      <td>12</td>\n      <td>12.0</td>\n      <td>50</td>\n      <td>0</td>\n    </tr>\n    <tr>\n      <td>Hong Kong</td>\n      <td>77</td>\n      <td>81.0</td>\n      <td>50</td>\n      <td>2</td>\n    </tr>\n  </tbody>\n</table>'